### Introduction/Business Problem

The city of Barcelona is one of the most visited cities in the world.
It is a large city with many robust neighborhoods and areas of interest.
For a traveler to the city, it can be difficult to maximize one's time and see the great things the city has to offer.
In order to provide the best possible experience in the city, we will compare different neighborhoods within the city to 
determine an optimal vacation. A traveler can use the information to determine which neighborhood to stay in based on
things like food, shopping, nightlife, and attractions.

In [1]:
!conda install -c conda-forge geopy --yes from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

usage: conda [-h] [-V] command ...
conda: error: unrecognized arguments: from geopy.geocoders import Nominatim
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.3.0               |        py36_1001         533 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         631 KB

The following NEW packages will be INSTALLED:

    altair:  2.3.0-py36_1001 conda-forge
    branca:  0.3.1-py_0      conda-forge
    folium:  0.5.0-py_0      co

### Project Description

In this project, we will use Foursquare to identify the must-do items in Barcelona. A user will be able to determine where to go in each neighborhood, what are some of the must-see attractions in the city, and where the best places to eat are. K-means clustering will be used to compare different neighborhoods within the city. 

Foursquare is a technology company that uses location intelligence to build meaningful consumer experiences and business solutions. We will use location data to find recommendations in different neighborhoods of Barcelona. 

Each neighborhood is defined as a district. The geospatial JSON file will be used to determine the latitude and longitude of each district. A call to the Foursquare API will get information for each of the different districts. K-means clustering will be used to examine the differences between neighborhoods. 

### Python packages used in the analysis:
Pandas 
NumPy
JSON
Geopy 
Requests
Matplotlib 
Sklearn 
Folium
IPython

In [6]:
#Get barcelona JSON data for each district

!wget -q -O 'bc_data.json' https://cdn.rawgit.com/martgnz/bcn-geodata/master/districtes/districtes_geo.json
print('Data downloaded!')

Data downloaded!


In [7]:
import json # library to handle JSON files
with open('bc_data.json') as json_data:
    bc_data = json.load(json_data)

In [8]:
bc_data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'C_Distri': '01',
    'N_Distri': 'Ciutat Vella',
    'Homes': 53968,
    'Dones': 48379,
    'Area': 4368464.78676,
    'Perim': 21035.207261,
    'Coord_X': 431616.77399,
    'Coord_Y': 4581563.51088,
    'WEB_1': 'http://www.bcn.cat/ciutatvella',
    'WEB_2': 'http://www.bcn.cat/estadistica/catala/dades/inf/anuari/index.htm',
    'WEB_3': 'http://www.bcn.cat/estadistica/catala/dades/guiadt01/index.htm'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[2.182389633367708, 41.39142593401981],
      [2.18346004924504, 41.39061092618033],
      [2.18460019085951, 41.389755039398544],
      [2.18686574382265, 41.39140848272813],
      [2.187942429966478, 41.390569517325424],
      [2.189183853611451, 41.389618576188205],
      [2.192556711542963, 41.387034739609526],
      [2.192965382371302, 41.38672944767718],
   

In [46]:
import os
os.getcwd()

'/resources/labs/DP0701EN'

In [55]:
neighborhoods  = pd.read_csv('bcn districts.csv')
neighborhoods.head()

,district,name,lat,long
0,1,Ciutat Vella,41.3809,2.1677
1,2,Eixample,41.3918,2.1642
2,3,Sants-Montjuoc,41.3561,2.1499
3,4,Les Corts,41.3835,2.1168
4,5,Sarria-Sant Gervasi,41.3993,2.1342


In [51]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [53]:
address = 'Barcelona'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Barcelona are 41.3828939, 2.1774322.


## Define Foursquare credentials

In [54]:
CLIENT_ID = 'CRZDBMKYTAPI4MVWPJ5Y5VA45VOTLMOZ13GWZY2VYHNPGA25' # your Foursquare ID
CLIENT_SECRET = 'GFLN1HBEMIF5221MDGZMRTW01W4S4KFVWYFWWBM1YWHL5QKH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CRZDBMKYTAPI4MVWPJ5Y5VA45VOTLMOZ13GWZY2VYHNPGA25
CLIENT_SECRET:GFLN1HBEMIF5221MDGZMRTW01W4S4KFVWYFWWBM1YWHL5QKH


In [59]:
# create map of Barcelona using latitude and longitude values
map_bcn = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(neighborhoods['lat'], neighborhoods['long'], neighborhoods['district']):
    label = '{}'.format( district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)  
    
map_bcn

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
LIMIT = 100
bcn_venues = getNearbyVenues(names=neighborhoods['name'],
                                   latitudes=neighborhoods['lat'],
                                   longitudes=neighborhoods['long']
                                  )


Ciutat Vella
Eixample
Sants-Montjuoc
Les Corts
Sarria-Sant Gervasi
Gracia
Horta-Guinardu
Nou Barris
Sant Andreu
Sant Marti


In [65]:
print(bcn_venues.shape)
bcn_venues.head()

(525, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ciutat Vella,41.3809,2.1677,Carlota Akaneya,41.382349,2.168812,Japanese Restaurant
1,Ciutat Vella,41.3809,2.1677,Llop,41.381849,2.169511,Restaurant
2,Ciutat Vella,41.3809,2.1677,A Tu Bola,41.380096,2.169054,Tapas Restaurant
3,Ciutat Vella,41.3809,2.1677,Caravelle,41.382337,2.168818,Restaurant
4,Ciutat Vella,41.3809,2.1677,NØMAD Every Day,41.382055,2.166349,Coffee Shop


In [66]:
bcn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ciutat Vella,100,100,100,100,100,100
Eixample,100,100,100,100,100,100
Gracia,100,100,100,100,100,100
Horta-Guinardu,5,5,5,5,5,5
Les Corts,23,23,23,23,23,23
Nou Barris,15,15,15,15,15,15
Sant Andreu,74,74,74,74,74,74
Sant Marti,54,54,54,54,54,54
Sants-Montjuoc,7,7,7,7,7,7


In [67]:
print('There are {} uniques categories.'.format(len(bcn_venues['Venue Category'].unique())))

There are 138 uniques categories.


In [68]:
# one hot encoding
bcn_onehot = pd.get_dummies(bcn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bcn_onehot['Neighborhood'] = bcn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bcn_onehot.columns[-1]] + list(bcn_onehot.columns[:-1])
bcn_onehot = bcn_onehot[fixed_columns]

bcn_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Ciutat Vella,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ciutat Vella,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ciutat Vella,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ciutat Vella,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ciutat Vella,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
bcn_grouped = bcn_onehot.groupby('Neighborhood').mean().reset_index()
bcn_grouped
bcn_grouped.shape



(10, 139)

In [71]:
num_top_venues = 5

for hood in bcn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bcn_grouped[bcn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ciutat Vella----
                      venue  freq
0                       Bar  0.08
1              Cocktail Bar  0.07
2        Spanish Restaurant  0.06
3          Tapas Restaurant  0.06
4  Mediterranean Restaurant  0.05


----Eixample----
              venue  freq
0             Hotel  0.14
1  Tapas Restaurant  0.06
2          Boutique  0.05
3    Clothing Store  0.04
4            Bakery  0.04


----Gracia----
              venue  freq
0              Café  0.08
1  Tapas Restaurant  0.07
2             Plaza  0.07
3      Cocktail Bar  0.05
4               Bar  0.05


----Horta-Guinardu----
               venue  freq
0              Trail   0.4
1            Dog Run   0.2
2               Farm   0.2
3               Park   0.2
4  Accessories Store   0.0


----Les Corts----
                 venue  freq
0            Nightclub  0.09
1         Tram Station  0.09
2         Soccer Field  0.09
3  Sporting Goods Shop  0.09
4   Italian Restaurant  0.04


----Nou Barris----
                 venue  f

In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['name'] = bcn_grouped['Neighborhood']

for ind in np.arange(bcn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bcn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,Bar,Cocktail Bar,Spanish Restaurant,Tapas Restaurant,Mediterranean Restaurant,Bookstore,Restaurant,Vegetarian / Vegan Restaurant,Hotel,Pub
1,Eixample,Hotel,Tapas Restaurant,Boutique,Spanish Restaurant,Mediterranean Restaurant,Clothing Store,Bakery,Bookstore,Furniture / Home Store,Vegetarian / Vegan Restaurant
2,Gracia,Café,Plaza,Tapas Restaurant,Cocktail Bar,Bar,Asian Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Bookstore
3,Horta-Guinardu,Trail,Park,Dog Run,Farm,Falafel Restaurant,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Women's Store
4,Les Corts,Soccer Field,Nightclub,Sporting Goods Shop,Tram Station,Bowling Alley,Museum,Soccer Stadium,Sports Club,Garden,College Cafeteria
5,Nou Barris,Spanish Restaurant,Grocery Store,Sports Club,Asian Restaurant,Fried Chicken Joint,Park,Bar,Pharmacy,Pizza Place,Restaurant
6,Sant Andreu,Spanish Restaurant,Bar,Sandwich Place,Café,Tapas Restaurant,Bakery,Falafel Restaurant,Mediterranean Restaurant,Pizza Place,Coffee Shop
7,Sant Marti,Mediterranean Restaurant,Restaurant,Italian Restaurant,Indian Restaurant,Cocktail Bar,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,Hotel
8,Sants-Montjuoc,Bar,Spanish Restaurant,Road,Soccer Field,Gourmet Shop,Mediterranean Restaurant,Falafel Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market
9,Sarria-Sant Gervasi,Spanish Restaurant,Restaurant,Bar,Bakery,Café,Italian Restaurant,Japanese Restaurant,Train Station,Hotel,Deli / Bodega


In [83]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

bcn_grouped_clustering = bcn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 3, 4, 0, 2, 2, 1, 2], dtype=int32)

In [86]:
bcn_merged = neighborhoods

# add clustering labels
bcn_merged['Cluster Labels'] = kmeans.labels_

# merge bcn_grouped with toronto_data to add latitude/longitude for each neighborhood
bcn_merged = bcn_merged.join(neighborhoods_venues_sorted.set_index('name'), on='name')

bcn_merged.head() # check the last columns!

,district,name,lat,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ciutat Vella,41.3809,2.1677,2,Bar,Cocktail Bar,Spanish Restaurant,Tapas Restaurant,Mediterranean Restaurant,Bookstore,Restaurant,Vegetarian / Vegan Restaurant,Hotel,Pub
1,2,Eixample,41.3918,2.1642,2,Hotel,Tapas Restaurant,Boutique,Spanish Restaurant,Mediterranean Restaurant,Clothing Store,Bakery,Bookstore,Furniture / Home Store,Vegetarian / Vegan Restaurant
2,3,Sants-Montjuoc,41.3561,2.1499,2,Bar,Spanish Restaurant,Road,Soccer Field,Gourmet Shop,Mediterranean Restaurant,Falafel Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market
3,4,Les Corts,41.3835,2.1168,3,Soccer Field,Nightclub,Sporting Goods Shop,Tram Station,Bowling Alley,Museum,Soccer Stadium,Sports Club,Garden,College Cafeteria
4,5,Sarria-Sant Gervasi,41.3993,2.1342,4,Spanish Restaurant,Restaurant,Bar,Bakery,Café,Italian Restaurant,Japanese Restaurant,Train Station,Hotel,Deli / Bodega


In [88]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bcn_merged['lat'], bcn_merged['long'], bcn_merged['name'], bcn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [91]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 0, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Gracia,Café,Plaza,Tapas Restaurant,Cocktail Bar,Bar,Asian Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Bookstore


In [92]:
##Cluster 2
bcn_merged.loc[bcn_merged['Cluster Labels'] == 1, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Sant Andreu,Spanish Restaurant,Bar,Sandwich Place,Café,Tapas Restaurant,Bakery,Falafel Restaurant,Mediterranean Restaurant,Pizza Place,Coffee Shop


In [93]:
##Cluster 3
bcn_merged.loc[bcn_merged['Cluster Labels'] == 2, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,Bar,Cocktail Bar,Spanish Restaurant,Tapas Restaurant,Mediterranean Restaurant,Bookstore,Restaurant,Vegetarian / Vegan Restaurant,Hotel,Pub
1,Eixample,Hotel,Tapas Restaurant,Boutique,Spanish Restaurant,Mediterranean Restaurant,Clothing Store,Bakery,Bookstore,Furniture / Home Store,Vegetarian / Vegan Restaurant
2,Sants-Montjuoc,Bar,Spanish Restaurant,Road,Soccer Field,Gourmet Shop,Mediterranean Restaurant,Falafel Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market
6,Horta-Guinardu,Trail,Park,Dog Run,Farm,Falafel Restaurant,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Women's Store
7,Nou Barris,Spanish Restaurant,Grocery Store,Sports Club,Asian Restaurant,Fried Chicken Joint,Park,Bar,Pharmacy,Pizza Place,Restaurant
9,Sant Marti,Mediterranean Restaurant,Restaurant,Italian Restaurant,Indian Restaurant,Cocktail Bar,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tapas Restaurant,Hotel


In [94]:
##Cluster 4
bcn_merged.loc[bcn_merged['Cluster Labels'] == 3, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Les Corts,Soccer Field,Nightclub,Sporting Goods Shop,Tram Station,Bowling Alley,Museum,Soccer Stadium,Sports Club,Garden,College Cafeteria


In [95]:
##Cluster 5
bcn_merged.loc[bcn_merged['Cluster Labels'] == 4, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Sarria-Sant Gervasi,Spanish Restaurant,Restaurant,Bar,Bakery,Café,Italian Restaurant,Japanese Restaurant,Train Station,Hotel,Deli / Bodega
